## Multilayer Perceptron (MLP)

- Camadas intermediárias
- Arquitetura feedforward
- aprendizagem por retropropagação (backpropagation)

<img src="images/MLP.png">

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
# importando dataset
dataset = pd.read_csv("frutas.csv")
dataset.head()

,ph,peso(kg),diametro(cm),fruta
0,2.980056,0.059878,8.303492,0
1,2.619839,0.059534,9.563558,0
2,2.983526,0.040029,8.921555,0
3,2.827375,0.046539,5.919455,0
4,2.979685,0.055912,9.392527,0


In [3]:
# extraindo características (x)
x = np.array(dataset.loc[:, ["ph", "peso(kg)", "diametro(cm)"]])
x

array([[ 2.98005606e+00,  5.98781498e-02,  8.30349159e+00],
       [ 2.61983873e+00,  5.95344254e-02,  9.56355780e+00],
       [ 2.98352600e+00,  4.00289555e-02,  8.92155540e+00],
       [ 2.82737493e+00,  4.65386710e-02,  5.91945531e+00],
       [ 2.97968528e+00,  5.59116887e-02,  9.39252729e+00],
       [ 2.99625546e+00,  5.36535158e-02,  7.63980642e+00],
       [ 3.30110957e+00,  4.72463817e-02,  8.08016791e+00],
       [ 3.42557126e+00,  5.15197758e-02,  6.19331529e+00],
       [ 2.78253640e+00,  3.49039649e-02,  6.81114797e+00],
       [ 3.55385466e+00,  3.74214658e-02,  7.66047212e+00],
       [ 3.52264228e+00,  7.03788412e-02,  8.67622907e+00],
       [ 2.84906888e+00,  6.15340605e-02,  7.99957211e+00],
       [ 2.88285328e+00,  4.35992624e-02,  8.89179211e+00],
       [ 3.28386152e+00,  5.01243049e-02,  9.01991579e+00],
       [ 2.74989284e+00,  6.18838284e-02,  7.91505903e+00],
       [ 3.47640560e+00,  4.55800761e-02,  9.87091082e+00],
       [ 2.31414672e+00,  4.36382278e-02

In [4]:
# extraindo resultado
d = np.array(dataset.loc[:,"fruta"])
d

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

#### Mapeando saídas:

Como nossa rede possui 3 neurônios de saída, vamos mapear qual será a saída esperada de cada neurônio para cada fruta

Fruta | Neurônio 0 | Neurônio 1 | Neurônio 2
---|---|---|---
Maçã  | 1 | 0 | 0
Limão | 0|1|0
Melão|0|0|1

In [5]:
# Mapeando classe para vetor
def vetorizar(d):
    saidas = []
    for fruta in d:
        if fruta == 0:
            saidas.append(np.array([1, 0, 0]))
        elif fruta == 1:
            saidas.append(np.array([0, 1, 0]))
        else:
            saidas.append(np.array([0, 0, 1]))

    return np.array(saidas)

# Mapeando vetor para classe
def classificar(vetor):
    classes = []
    for y in vetor:
        maximo = np.argmax(y)
        classes.append(maximo)
        
    return classes

In [6]:
d = vetorizar(d)

# Embaralhando os dados
shuffle = np.random.permutation(len(x))
x = x[shuffle]
d = d[shuffle]

In [7]:
# Separando dados de treino (80%) e teste (20%)
limite = int(len(x) * 0.8)

x_treino = x[0:limite]
x_teste = x[limite: ]

d_treino = d[0:limite]
d_teste = d[limite: ]

In [72]:
class MLP():
    
    def __init__(self, lr, e, neurons):
        """ Construtor """
        # Taxa de aprendizado 
        self.lr = lr
        
        # tolerância
        self.e = e
        
        # Quantidade de neurônios por camadas
        self.neurons = neurons
        
        
    def sigmoid(self, valor):
        '''Calcula a sigmoid de um valor'''
        return (1/(1+math.e**(-valor)))

    def sigmoid_deriv(self, valor):
        '''Calcula a derivada da função sigmoid'''
        sig = self.sigmoid(valor)
        return sig*(1 - sig)

    def activate(self, valor):
        '''Ativa as saídas do neurônio'''
        return self.sigmoid(valor)
    
    def deriv(self, valor):
        '''Calcular a derivada da função de ativação'''
        return self.sigmoid_deriv(valor)

    def evaluate(self, target, predicted):
        '''Calcula a diferença entre o valor real e o valor predito'''
        return (target - predicted)

    def predict(self, input_data, weights):
        '''Calcula a soma ponderada das entradas pelo peso'''
        return np.dot(input_data, weights).reshape(1, -1)
    
    
    def train(self, x, d):
        ''' 
        Definir aleatoriamente os pesos, o bias e o peso do bias
        Enquanto a diferença entre m mse_anterior e o mse_atual for maior que 'e' continua o processo 
        '''
        self.w1 = np.random.random((x.shape[1]+1,self.neurons[0]))
        self.w2 = np.random.random((self.neurons[0], self.neurons[1]))
        self.w3 = np.random.random((self.neurons[1], self.neurons[2]))
        
        epoch = 0
        last_mse = np.inf
        self.total_mse = []
        self.bias = -1
        
        while True:
            mse = 0
            for xi, target in zip(x,d):
                input_value = np.insert(xi, 0, self.bias)
                i1 = self.predict(input_value, self.w1)
                y1 = self.activate(i1)
                i2 = self.predict(y1, self.w2)
                y2 = self.activate(i2)
                i3 = self.predict(y2, self.w3)
                y3 = self.activate(i3)
                current_error = self.evaluate(target, y3)
                mse+=(current_error ** 2)

                delta3 = (target - y3) * self.deriv(i3)
                self.w3 += self.lr * np.dot(y2.T, delta3)

                delta2 = np.dot(delta3, self.w3.T) * self.deriv(i2)
                self.w2 += self.lr * np.dot(y1.T, delta2)

                delta1 = np.dot(delta2, self.w2.T) * self.deriv(i1)
                self.w1 += self.lr * np.dot(input_value.reshape(1, -1).T, delta1)

            mse = sum(mse[0]) / len(x)
            
            print(f"EPOCH: {epoch} - MSE: {mse} - |mse_ant - mse|: {abs(last_mse - mse)}")
            if abs(last_mse - mse) <= self.e:
                break
            
            self.total_mse.append(mse)
            last_mse = mse
            epoch += 1
            
            
    def test(self, x):
        ''' Dado uma lista de X, submete-os à rede'''
        results = []
        for xi in x:
            input_value = np.insert(xi, 0, self.bias)
            i1 = self.predict(input_value, self.w1)
            y1 = self.activate(i1)
            i2 = self.predict(y1, self.w2)
            y2 = self.activate(i2)
            i3 = self.predict(y2, self.w3)
            y3 = self.activate(i3)
            
            results.append(y3[0])
            
        return results

In [73]:
rede = MLP(lr = 0.01, e = 1e-7, neurons = [5, 4, 3])
rede.train(x = x_treino, d = d_treino)

EPOCH: 0 - MSE: 1.402931912495388 - |mse_ant - mse|: inf
EPOCH: 1 - MSE: 1.3323761618002878 - |mse_ant - mse|: 0.07055575069510023
EPOCH: 2 - MSE: 1.2575759279211276 - |mse_ant - mse|: 0.07480023387916024
EPOCH: 3 - MSE: 1.180346361863351 - |mse_ant - mse|: 0.07722956605777664
EPOCH: 4 - MSE: 1.102717754374056 - |mse_ant - mse|: 0.0776286074892949
EPOCH: 5 - MSE: 1.0270357074965117 - |mse_ant - mse|: 0.07568204687754432
EPOCH: 6 - MSE: 0.9560302120365827 - |mse_ant - mse|: 0.07100549545992907
EPOCH: 7 - MSE: 0.8924922089041686 - |mse_ant - mse|: 0.06353800313241409
EPOCH: 8 - MSE: 0.8385362980539077 - |mse_ant - mse|: 0.05395591085026086
EPOCH: 9 - MSE: 0.7949519135911033 - |mse_ant - mse|: 0.04358438446280444
EPOCH: 10 - MSE: 0.7611545808261394 - |mse_ant - mse|: 0.03379733276496388
EPOCH: 11 - MSE: 0.7356828329821874 - |mse_ant - mse|: 0.02547174784395201
EPOCH: 12 - MSE: 0.7168029001730208 - |mse_ant - mse|: 0.018879932809166555
EPOCH: 13 - MSE: 0.7029117598438166 - |mse_ant - mse|:

KeyboardInterrupt: 

In [ ]:
resultado = rede.test(x_teste)
resultado

[array([0.2909744 , 0.34852531, 0.36023722]),
 array([0.29175502, 0.34912876, 0.3606979 ]),
 array([0.290973  , 0.34852423, 0.36023632]),
 array([0.29125819, 0.34874317, 0.36041771]),
 array([0.29302352, 0.35012824, 0.36128034]),
 array([0.29258147, 0.34976553, 0.36119227]),
 array([0.29290064, 0.35001095, 0.36138542]),
 array([0.29145587, 0.34889573, 0.36053629]),
 array([0.29097317, 0.34852437, 0.36023642]),
 array([0.29143273, 0.34887732, 0.36052654]),
 array([0.29195612, 0.34928303, 0.36082392]),
 array([0.29139573, 0.34885042, 0.36049205]),
 array([0.29296592, 0.35007794, 0.36129408]),
 array([0.29149493, 0.34892886, 0.36053695]),
 array([0.29135727, 0.34881988, 0.36047538]),
 array([0.290976  , 0.34852654, 0.36023824]),
 array([0.29269746, 0.34985329, 0.36127311]),
 array([0.29142397, 0.34887107, 0.36051742])]

In [ ]:
d_teste

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0]])

In [ ]:
classes_resultado = classificar(resultado)
classes_esperado = classificar(d_teste)

In [ ]:
print(classes_resultado)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [ ]:
print(classes_esperado)

[2, 0, 2, 0, 1, 1, 1, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 0]


In [ ]:
def acuracia(real, predito):
    acertos = 0
    for i in range(len(real)):
        if real[i] == predito[i]:
            acertos+=1
    
    return acertos/len(real)

In [ ]:
acuracia(classes_esperado, classes_resultado)

0.2222222222222222